In [1]:
import EDS as eds

required_para = eds.required_parameters(    
    spray_number = [1, 2, 3],
    spray_moment = [30, 45, 60],
    spray_eff = [0.5, 0.5, 0.5],
    
    # for ["Susceptible", "Moderate", "Resistant"]
    p_opt = [7, 10, 14],
    rc_opt_par = [0.35, 0.25, 0.15],
    rrlex_par = [0.1, 0.01, .0001]
)

result = eds.calculate_disease_severity(    
    data_path = "Original_Code/Data.csv",
    info_path = "Original_Code/Info.csv",
    number_of_repeat_year = 4,       
    number_applications_list = [0],
    genetic_mechanistic_list = ["Susceptible"],    
    required_para = required_para,
    output_columns = ["SevMAX"],   
    path_result = None
)

# result.to_csv(f"results.csv", index=None)

result

/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["info_id"] = row[1]["ID"] + "_" + str(row[1]["year"]) + "_" + str(row[1]["planting_date"]) + "_" + str(row[1]["obs_planting_delta"]) + "_" + row[1]["Crop"]
/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["year"] = row[1]["year"]
/home/pooya/miniconda3/lib/python3.9/site-packages/EDS/location_data.py:44: Setti

,locationId,Date1,Date2,N_Days,latitude,longitude,number_applications,genetic_mechanistic,crop,SevMAX
0,ID_44.26405373_-92.9375238_2017_5/3/2017_129_Soy,20170503,20170909,129,44.264054,-92.937524,0,Susceptible,Soy,8.641549e-01
1,ID_44.740789_-91.2186815_2016_5/15/2016_73_Corn,20160515,20160727,73,44.740789,-91.218682,0,Susceptible,Corn,3.540606e-07
2,ID_44.97603894_-95.21102976_2017_5/5/2017_98_Corn,20170505,20170811,98,44.976039,-95.211030,0,Susceptible,Corn,6.978343e-01
3,ID_44.97603894_-95.21102976_2019_5/5/2019_89_Corn,20190505,20190802,89,44.976039,-95.211030,0,Susceptible,Corn,3.766721e-01


# Step to Step Run

In [ ]:
import datetime
import itertools
import math
import numpy as np
import pandas as pd

from Create_Package_Original.EDS.required_parameters import *
from Create_Package_Original.EDS.calculate_disease_severity import *



required_para = required_parameters(    
    spray_number = [1, 2, 3],
    spray_moment = [30, 45, 60],
    spray_eff = [0.5, 0.5, 0.5],
    
    # for ["Susceptible", "Moderate", "Resistant"]
    p_opt = [7, 10, 14],
    rc_opt_par = [0.35, 0.25, 0.15],
    rrlex_par = [0.1, 0.01, .0001]
)


result = calculate_disease_severity(    
    data_path = "Original_Code/data/Data.csv",
    info_path = "Original_Code/data/Info.csv",
    add_year = True,
    
    number_applications_list = [0, 1, 2, 3],
    genetic_mechanistic_list = ["Susceptible", "Moderate", "Resistant"],
    
    required_para = required_para,
    
    path_result = None
)


result.to_csv(f"results.csv", index=None)